In [1]:
from datetime import datetime
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import math
import warnings

/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def read_in_tables(n=78):
    df_list = {}
    for i in range(1, n):

        table1_name = 'test_df' + str(i)
        table2_name = 'train_df' + str(i)

        df = pd.read_csv(f'../datasets/training/SARIMA/by_area/ml_{i}.csv', parse_dates=['date'])

        globals()[table1_name] = df[(df.date <= pd.to_datetime('2019-12-31')) & (df.date >= pd.to_datetime('2019-01-01'))]
        globals()[table2_name] = df[(df.date < pd.to_datetime('2019-01-01')) & (df.date > pd.to_datetime('2016-01-01'))]

        globals()[table1_name].fillna(0, inplace=True)
        globals()[table2_name].fillna(0, inplace=True)

        globals()[table1_name].set_index('date', inplace=True)
        globals()[table2_name].set_index('date', inplace=True)

        df_list[table1_name] = globals()[table1_name]
        df_list[table2_name] = globals()[table2_name]

    return df_list

In [3]:
df_list = read_in_tables()
len(df_list)

/var/folders/5d/fdmfhfr55d5f2r62ltcl50vh0000gn/T/ipykernel_45843/3740099932.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[table1_name].fillna(0, inplace=True)
/var/folders/5d/fdmfhfr55d5f2r62ltcl50vh0000gn/T/ipykernel_45843/3740099932.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[table2_name].fillna(0, inplace=True)
/var/folders/5d/fdmfhfr55d5f2r62ltcl50vh0000gn/T/ipykernel_45843/3740099932.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

154

In [4]:
def dickey_fuller_test(df, exempt_cols=['date'], sig_level=0.05):
    """
    Perform Dickey-Fuller test for stationarity on all columns in a DataFrame.

    Parameters
    ----------
    df : DataFrame
    Input time series data 
    exempt_cols : list, default ['date']
    Columns to skip testing
    sig_level : float, default 0.05 
    Significance level for testing

    Returns
    -------
    results : dict
    Dictionary containing test results for each column 
    """
    # Dictionary to store results
    results = {} 

    # Loop through columns
    for col in df.columns:

        # Skip exempt columns
        if col in exempt_cols:
            continue

        # Print progress  
        print(f"{datetime.now()}: Testing {col} for stationarity...")

        try:
            # Perform ADF test
            adf_result = adfuller(df[col])
            
            # Store results    
            results[col] = {
                'adf_stat': adf_result[0],
                'pvalue': adf_result[1],
                'sig_level': sig_level,
                'result': adf_result[1] < sig_level
            }

            # Print conclusions
            pvalue = results[col]['pvalue']
            sig = results[col]['sig_level']
            
            stationary = pvalue < sig  
            print(f"{col} stationary: {stationary} (p-value: {pvalue})")
        
        except Exception as e:
            # Handle errors
            print(f"Error testing {col}: {str(e)}. Skipping.")
            results[col] = {
                'adf_stat': None,
                'pvalue': None,
                'sig_level': sig_level
            }
        
    return results

In [5]:
def find_auto_arima(series, adf_result, max_p=5, max_q=5, max_d=1, stepwise=True, ic='aic'):
    """
    Search for optimal ARIMA parameters for a time series.

    Parameters
    ----------
    series : pd.Series
        The time series data.
    adf_result : bool
        Whether the time series data is stationary or not (ADF test result).
    max_p, max_q : int
        Maximum AR (p) and MA (q) orders to check.
    max_d : int
        Maximum number of differences allowed.
    stepwise : bool
        Whether to use stepwise model selection.

    Returns
    -------
    tuple
        A tuple containing the best ARIMA model order (p, d, q).
    """
    now = datetime.now()

    if adf_result:
        #print(f"{now} - Fitting models...")
        mods = pm.auto_arima(series, start_p=0, start_q=0, max_p=max_p, max_q=max_q, seasonal=True,
                            max_d=max_d, stepwise=stepwise, suppress_warnings=True, information_criterion=ic)
    else:
        #print(f"{now} - Fitting models...")
        mods = pm.auto_arima(series, start_p=0, start_q=0, max_p=max_p, max_q=max_q, seasonal=False,
                            max_d=max_d, stepwise=stepwise, suppress_warnings=True, information_criterion=ic)

    #print(f"{now} - Selecting best model...")
    best_mod = mods.order

    #print(f"{now} - Best model found: {best_mod}")
    return best_mod

In [14]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

    test_result = ARIMA_model(test_df1, 'violent', 4, 0, 0, True)

2023-10-24 23:46:41.409596: Data for column "violent" is stationary. Fitting SARIMA model.


/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.06131D-01    |proj g|=  4.28024D-02


 This problem is unconstrained.



At iterate    5    f=  4.98939D-01    |proj g|=  9.82183D-04

At iterate   10    f=  4.98937D-01    |proj g|=  1.67253D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     10     13      1     0     0   1.673D-06   4.989D-01
  F =  0.49893713216479574     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [6]:
final_adf = {}
for i in range(1, 78):
    table = 'test_df' + str(i)
    final_adf[table] = dickey_fuller_test(globals()[table])
    print(f'Table {i} done')

2023-10-24 22:01:47.524633: Testing non-violent for stationarity...
non-violent stationary: True (p-value: 1.225437653526885e-29)
2023-10-24 22:01:47.799366: Testing violent for stationarity...
violent stationary: True (p-value: 0.0)
2023-10-24 22:01:48.076040: Testing train_rides for stationarity...
train_rides stationary: True (p-value: 7.084541545618042e-25)
2023-10-24 22:01:48.342086: Testing bike_rides for stationarity...
bike_rides stationary: True (p-value: 3.450597478943833e-05)
2023-10-24 22:01:48.598466: Testing lighting for stationarity...
lighting stationary: False (p-value: 0.06504181533166485)
2023-10-24 22:01:48.858290: Testing vacant_buildings for stationarity...
vacant_buildings stationary: False (p-value: 0.9263468991230118)
Table 1 done
2023-10-24 22:01:49.081853: Testing non-violent for stationarity...
non-violent stationary: True (p-value: 2.7824026642043217e-28)
2023-10-24 22:01:49.331171: Testing violent for stationarity...
violent stationary: True (p-value: 0.0)

In [7]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

    final_aa = {}
    for table_name, cols in final_adf.items():
        print(f'{datetime.now()} - Table {table_name} started')
        table_results = {}
        for col in cols:
            print(f'{datetime.now()} - Column {col} started')
            if cols[col]['pvalue'] == None:
                table_results[col] = None
            else:
                table_results[col] = find_auto_arima(globals()[table_name][col], cols[col]['result'])
            print(f'{datetime.now()} - Column {col} done')
        
        final_aa[table_name] = table_results
        print(f'{datetime.now()} - Table {table_name} done')

2023-10-24 22:03:22.505205 - Table test_df1 started
2023-10-24 22:03:22.505525 - Column non-violent started
2023-10-24 22:03:28.079415 - Column non-violent done
2023-10-24 22:03:28.081858 - Column violent started
2023-10-24 22:03:36.896717 - Column violent done
2023-10-24 22:03:36.896801 - Column train_rides started
2023-10-24 22:03:37.430924 - Column train_rides done
2023-10-24 22:03:37.432466 - Column bike_rides started
2023-10-24 22:03:48.514503 - Column bike_rides done
2023-10-24 22:03:48.514579 - Column lighting started
2023-10-24 22:03:49.566143 - Column lighting done
2023-10-24 22:03:49.566220 - Column vacant_buildings started
2023-10-24 22:03:51.027619 - Column vacant_buildings done
2023-10-24 22:03:51.027691 - Table test_df1 done
2023-10-24 22:03:51.027697 - Table test_df2 started
2023-10-24 22:03:51.027700 - Column non-violent started
2023-10-24 22:04:07.105250 - Column non-violent done
2023-10-24 22:04:07.105332 - Column violent started
2023-10-24 22:04:15.241989 - Column vi

In [8]:
len(final_aa)

77

In [10]:
final_aa

{'test_df1': {'non-violent': (2, 0, 0),
  'violent': (4, 0, 0),
  'train_rides': (0, 1, 0),
  'bike_rides': (0, 1, 2),
  'lighting': (0, 1, 0),
  'vacant_buildings': (0, 1, 0)},
 'test_df2': {'non-violent': (5, 1, 0),
  'violent': (1, 0, 1),
  'train_rides': None,
  'bike_rides': (0, 1, 1),
  'lighting': (0, 1, 0),
  'vacant_buildings': (0, 1, 0)},
 'test_df3': {'non-violent': (3, 0, 0),
  'violent': (4, 1, 1),
  'train_rides': (1, 0, 0),
  'bike_rides': (2, 1, 0),
  'lighting': (0, 1, 0),
  'vacant_buildings': (0, 1, 0)},
 'test_df4': {'non-violent': (5, 1, 0),
  'violent': (1, 0, 1),
  'train_rides': (1, 0, 0),
  'bike_rides': (0, 1, 2),
  'lighting': (0, 1, 0),
  'vacant_buildings': (0, 1, 0)},
 'test_df5': {'non-violent': (1, 0, 1),
  'violent': (5, 1, 1),
  'train_rides': (1, 0, 0),
  'bike_rides': (0, 1, 3),
  'lighting': (0, 1, 0),
  'vacant_buildings': (0, 1, 0)},
 'test_df6': {'non-violent': (2, 1, 2),
  'violent': (0, 1, 1),
  'train_rides': (0, 1, 0),
  'bike_rides': (0, 1, 

In [22]:
final_adf['test_df1']['non-violent']['result']

True

In [59]:
def ARIMA_model(df, col, p, d, q, stationary_state, m=24, test_df=None):
    """
    Fit a SARIMA or ARIMA model based on the stationarity state and make predictions.

    Parameters:
    - df: DataFrame containing the time series data.
    - col: Name of the column to model.
    - p, d, q: Parameters for the ARIMA component.
    - stationary_state: True if data is stationary, False otherwise.
    - m: Seasonal period for SARIMA model.
    - forecast_steps: Number of steps to forecast (optional).
    - test_df: DataFrame for testing and comparing predictions (optional).

    Returns:
    - results: Fitted SARIMA or ARIMA model.
    - predictions_df: DataFrame with predictions and comparisons (if test_df provided).
    - loss: Mean squared error loss (if test_df provided).
    """
    if stationary_state:
        print(f'{datetime.now()}: Data for column "{col}" is stationary. Fitting SARIMA model.')
        model = SARIMAX(df[col],
                        order=(p, d, q),
                        seasonal_order=(p, d, q, m))
    else:
        print(f'{datetime.now()}: Data for column "{col}" is not stationary. Fitting ARIMA model instead.')
        model = ARIMA(df[col], order=(p, d, q))

    results = model.fit()
    
    # Default to one year if no forecast steps are specified
    n_forecast = 24 * 365  # Assuming hourly data for a year
    forecast = results.get_forecast(steps=n_forecast)
    forecast_mean = forecast.predicted_mean
    
    predictions_df = pd.DataFrame({'Actual': df[col], 'Forecast': forecast_mean})
    
    loss = None
    if test_df is not None:
        loss = ((test_df[col] - forecast_mean) ** 2).mean()
    
    return results, predictions_df, loss

In [60]:
def run_ARIMA_models(final_aa, final_adf):
    """
    Run ARIMA models for multiple tables and columns.

    Parameters:
    - final_aa: Dictionary containing ARIMA model parameters for each table and column.
    - final_adf: Dictionary containing ADF test results for each table and column.

    Returns:
    - ARIMA_results: Dictionary containing ARIMA model results.
    """
    ARIMA_results = {}

    for table_index, table in enumerate(final_aa.keys()):
        target_table = globals()[table]
        col_results = {}

        for col in final_aa[table]:
            if final_adf[table][col]['pvalue'] is None:
                d = {'col': col, 'model': None, 'predictions_df': target_table[col], 'loss': None}
            else:
                p, d, q = final_aa[table][col]
                state = final_adf[table][col]['result']
                test_table = 'test_df' + str(table_index + 1)
                results, predictions_df, loss = ARIMA_model(target_table, col, p, d, q, state, 24, globals()[test_table])
                d = {'col': col, 'model': results, 'predictions_df': predictions_df, 'loss': loss}

            col_results[col] = d
            print(f'{col} column finished')

        ARIMA_results[table] = col_results
        print(f'{table} table finished')

    return ARIMA_results

In [61]:
ARIMA_results = run_ARIMA_models(final_aa, final_adf)

2023-10-26 00:19:05.073440: Data for column "non-violent" is stationary. Fitting SARIMA model.


/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.60293D-01    |proj g|=  7.66776D-02


 This problem is unconstrained.



At iterate    5    f=  8.48385D-01    |proj g|=  2.17608D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      8     11      1     0     0   2.198D-05   8.484D-01
  F =  0.84838460918049441     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


...

 This problem is unconstrained.



At iterate    5    f=  2.95026D+00    |proj g|=  1.19464D-02

At iterate   10    f=  2.92951D+00    |proj g|=  1.47926D-02

At iterate   15    f=  2.92544D+00    |proj g|=  8.26246D-04

At iterate   20    f=  2.92481D+00    |proj g|=  1.06303D-02

At iterate   25    f=  2.92401D+00    |proj g|=  1.68031D-03

At iterate   30    f=  2.92391D+00    |proj g|=  5.62367D-04

At iterate   35    f=  2.92390D+00    |proj g|=  6.26137D-04

At iterate   40    f=  2.92390D+00    |proj g|=  2.95635D-04

At iterate   45    f=  2.92390D+00    |proj g|=  3.27533D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     46     79   


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/evanmeyer/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


bike_rides column finished
2023-10-26 00:36:07.349578: Data for column "lighting" is not stationary. Fitting ARIMA model instead.


NotImplementedError: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been removed in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and statsmodels.tsa.SARIMAX.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained. It also offers alternative specialized
parameter estimators.
